Unzip the image files, e.g. from your Drive.

In [ ]:
!zip -r /content/drive/MyDrive/2024-01-09-Bauernproteste/2024-01-09-Images-Clean.zip media

Import the CSV-File exported from Gephi. Set `sample_size` to your desired number, I recommend a low number, e.g. 5.

In [54]:
import pandas as pd

gephi_file = "/content/drive/MyDrive/2024-01-09-Bauernproteste/2024-01-11-Google-Vision-Graph-w-modclasses.csv"  #@param {type:"string"}
sample_size = 5 
gephi_df = pd.read_csv(gephi_file)

Render the Sample: Hit run for the next cell to create an HTML view of image classifications. The HTML will also be saved to file, check the files in the left pane for a file named `{formatted_date}-Gephi-Mod-Classes-Visualisation.html` to download the document to your computer. The file includes the `base64` encoded images.

In [ ]:
from IPython.display import HTML, display
import datetime
import base64

def get_base64_encoded_image(image_path):
    with open(image_path, "rb") as img_file:
        return base64.b64encode(img_file.read()).decode('utf-8')


# Include the Bootstrap CSS
bootstrap_link = '<link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@5.2.3/dist/css/bootstrap.min.css" integrity="sha384-rbsA2VBKQhggwzxH7pPCaAqO46MgnOM80zW1RWuH61DGLwZJEdK2Kadq2F9CUG65" crossorigin="anonymous">'

# Create an empty list to store the HTML content
html_content = []

# Group the DataFrame by image_type
filtered_gephi_df = gephi_df[gephi_df['attr_type'] == "Image_BaseName"]
grouped = filtered_gephi_df.groupby("modularity_class")

# Start the grid container
html_content.append("<div class='container-fluid mt-3'>")

# Iterate over image_type groups
for image_type, group_df in grouped:
    # Create a header for the image_type group
    html_content.append(f"<h2>Modularity Class #{image_type}</h2>")
    html_content.append("<div class='row row-cols-1 row-cols-md-6 g-10'>")

    group_df_sample = group_df.sample(sample_size, replace=True)
    # Iterate over rows in the group
    for _, row in group_df_sample.iterrows():
        image_path = f"media/images/{row['image']}"
        base64_image = get_base64_encoded_image(image_path)
        image_tag = f'<img src="data:image/jpeg;base64,{base64_image}" class="card-img-top">'

        card_content = f"""
        <div class='col-lg-2 col-md-4 col-sm-6 mb-4'>
            <div class='card h-100'>
                {image_tag}
            </div>
        </div>
        """
        html_content.append(card_content)

    # Close the row and container for the image_type group
    html_content.append("</div>")

# Close the overall container
html_content.append("</div>")

css = """
<style>
@media print {
    h2 {
      break-before: page;
    }
    .grid-container {
        display: flex;
        flex-wrap: wrap;
        justify-content: space-between;
        width: 297mm; /* A4 landscape width */
        height: 210mm; /* A4 landscape height */
        padding: 5mm; /* Padding around the edges */
        gap: 3mm; /* Gap between grid items */
    }
    .image-entry {
        flex: 1 1 calc(16.66% - 3mm); /* 6 images per row with a little space between */
        box-sizing: border-box;
        text-align: center;
        page-break-inside: avoid;
        border: 1px solid #ddd; /* A subtle border around each image entry for clarity */
        padding: 2mm; /* Padding inside each grid item */
    }
    .image-entry h2 {
        font-size: 11pt; /* Adjusted font size */
        margin: 2mm 0; /* Reduced margin around the text */
    }
    img {
        max-width: 100%;
        max-height: 60mm; /* Adjusted image height to fit 3 rows */
        display: block;
        margin: 0 auto;
    }
}
</style>
"""

final_html = f"{bootstrap_link}\n{css}\n{' '.join(html_content)}"

display(HTML(final_html))


# Get the current date and time
current_date = datetime.datetime.now()

# Format the date as YYYY-MM-DD
formatted_date = current_date.strftime('%Y-%m-%d')


# Generate the HTML file name with the current date
html_file_name = f"{formatted_date}-Gephi-Mod-Classes-Visualisation.html"


# Write the HTML to a file
with open(html_file_name, "w") as file:
    file.write(final_html)
